# Flight Data Files to CSV
Here we create the CSV file of per-flight data to import into our WordPress website.  By creating flights (technically, WordPress posts of `post_type` = `flight`) from CSV, I've been able to iterate dozens of times, trying different datatypes, etc.  New fields can be easily added, again, without having to enter data into forms by hand.  **The saga of WordPress and datatypes is far too long to describe here, but is very imporant you understand them...**

Per-flight metadata (flight date, altitude, pilots, etc.) came mostly from Jackie's spreadsheet, while data generated and collected during each flight was gathered from several different ad-hoc locations, including Morgan's SVN repo and https://perlanproject.cloud/data, and organized into one directory per flight, with standardized filenames.  For example, the KML file for Flight 0065 is named `/data/Flights/0069/Flt0069.kml`.  

NOTES:

- The web hostname must be prepended to make a full-path URI, e.g., `http://localhost/data/Flights/0069/Flt0069.kml`.  For development, hostname is `http://localhost`.  When the site is deployed, the hostname is changed to reflect this, either manually (if you're a stud) or via WordPress plugin.  Using our example, `https://perlanproject.org/data/Flights/0069/Flt0069.kml`

- The `data` directory lives at the top level (root) of the website, or more precisely, is *accessible* from the root; it is actually symlinked so it can be backed up in a Git repo using Large File Support (LFS), as there are some big honkin' files.

- We actually read *and* write `flights.csv` and then *write* `flights-toolset.csv`, the latter being the file we import into WordPress **using the Perlan plugin** to create Flights (`post_type == 'flight'`) as implemented by the Toolset plugin.  Toolset is a paid (tho inexpensive) WordPress plugin that implements the custom data types, custom fields, custom post types, etc. we need.  WordPress itself is **very** bare-bones, as it was designed as a blogging platform, not a Content Management System (CMS).  At least that the excuse they give.  IMHO, WordPress would be a **lot** more stable, more powerful, more usable, and all-around have much better karma, had the WordPress developers decided to add all these missing pieces directly into the platform at some point, rather than let the hodge-podge we have today happen.  But nooooooooo....

- We use the Perlan plugin for importing CSV because the free one recommended by Toolset, "CSV Importer" has not been maintained and was buggy.  The paid CSV importers were expensive ($300) and did more than we needed.  Besides, I had a CSV import utility written into the Perlan plugin, anyway.  The code is simple, and gives us a window into the process, too, esp. when run under the XDebug debugger via Eclipse.

- Since we're concerned with CSV files, which are just text, we must ensure that we use empty strings ("") and **not** Python None or NaNs for any columns with empty values.

In [1]:
import os
from datetime import datetime as dt
import numpy as np
import pandas as pd
import path_utils as pu

In [2]:
root = "/Users/jdm/workbench/Perlan" # YMMV
os.chdir(root)

In [3]:
!pwd

/Users/jdm/workbench/Perlan


In [4]:
!ls

Controlled.svn                    data_website.202105026_1010
Perlan Encore Fellowship          data_website.202105026_1010.zip
PerlanProject-2020-07-07T19-16-38 data_website.drupaled.broken
Science.git                       data_website.duplicator.archives
TRASH_LATER                       perlanproject.org
Toolset                           pods
assets                            pods.old
clippings                         tmp
data                              windField
data website plan.ooutline        wp-config.php.save
data_website                      wp_options__wpcf_fields


# Read CSV

We expect `flights.csv` to have the per-flight metadata already.  Originally it was gleaned from `flights.xlsx` but now has been further modified.

After reading `flights.csv` we walk the directory `data/Flights` to find data files and add those names to the CSV.

In [5]:
svn_root  = 'Controlled.svn/Systems/Data Network Logs/'
data_root = 'data/'
flights_root = data_root + 'Flights'
ballons_root = data_root + 'Soundings'

In [6]:
csv = pd.read_csv(f"{data_root}/flights.csv")

In [7]:
csv.columns

Index(['post_type', 'post_title', 'post_content', 'post_excerpt',
       'flight_number', 'takeoff_date_local', 'takeoff_time_local_HM',
       'takeoff_airport', 'takeoff_city', 'landing_date_local',
       'landing_time_local_HM', 'landing_airport', 'landing_city',
       'flight_duration_hours', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_adp', 'data_imu', 'data_kml', 'data_uv'],
      dtype='object')

## Initial Cleanup


In [8]:
# Create post_title.  e.g., if flight_number == 69, post_title = "Flight 0069"
csv['post_title'] = csv.flight_number.apply(lambda s: f"Flight {s:04}")

In [9]:
# Zero-out the columns we populate by walking the directory tree to find files
csv['data_ac'] = ""
csv['data_uv'] = ""
csv['data_kml'] = ""
csv['data_adp'] = ""
csv['data_imu'] = ""

### Deal with missing/bad data

In [10]:
# delete any null rows
csv.dropna(how='all', inplace=True)

In [11]:
# Convert all Nan and None to ""
# This isn't usually needed, as df.to_csv() usually works, but just to be completely sure we get what we need...
csv = csv.fillna(value="")

In [12]:
# Delete 'Unnamed' columns - this happens if CSV is saved via df.to_csv(index=True)
# This shouldn't happen, of course, and it's harmless, but annoying.
for col in csv.columns:
    match = 'Unnamed'
    if col[:len(match)] == match:
        print(f"Deleting junk column {col}")
        del csv[col]

### Data Types

In [13]:
# ensure correct types - int columns can become float if any missing data
csv['flight_number'] = csv.flight_number.astype(int, copy=False)

In [14]:
# Make sure HM times are zero-padded 4-digit strings
for col in ['takeoff_time_local_HM', 'landing_time_local_HM']:
    csv[col] = csv[col].astype(str, copy=False)
    csv[col] = csv[col].apply(lambda s: f"{int(s):04}")

## Sanity Check: Input
Look these over to make sure everything looks OK.

In [15]:
csv.head()

,post_type,post_title,post_content,post_excerpt,flight_number,takeoff_date_local,takeoff_time_local_HM,takeoff_airport,takeoff_city,landing_date_local,...,release_altitude_feet,maximum_altitude_feet,maximum_gps_altitude_feet,pilot_front,pilot_rear,data_ac,data_adp,data_imu,data_kml,data_uv
0,flight,Flight 0001,First Flight,First Flight,1,2015-09-23,0818,KRDM,"Redmond, OR",2015-09-23,...,8100,8100,,Jim Payne,Morgan Sandercock,,,,,
1,flight,Flight 0002,flight testing,flight testing,2,2016-01-15,1307,KMEV,"Minden, NV",2016-01-15,...,10800,10800,,Jim Payne,Miguel Iturmendi,,,,,
2,flight,Flight 0003,,,3,2016-01-15,1420,KMEV,"Minden, NV",2016-01-15,...,10700,10700,,Jim Payne,Miguel Iturmendi,,,,,
3,flight,Flight 0004,,,4,2016-01-27,1423,KMEV,"Minden, NV",2016-01-27,...,7600,7600,,Jim Payne,,,,,,
4,flight,Flight 0005,,,5,2016-01-27,1525,KMEV,"Minden, NV",2016-01-27,...,8700,8700,,Jim Payne,,,,,,


In [16]:
csv.dtypes

post_type                     object
post_title                    object
post_content                  object
post_excerpt                  object
flight_number                  int64
takeoff_date_local            object
takeoff_time_local_HM         object
takeoff_airport               object
takeoff_city                  object
landing_date_local            object
landing_time_local_HM         object
landing_airport               object
landing_city                  object
flight_duration_hours        float64
release_altitude_feet          int64
maximum_altitude_feet          int64
maximum_gps_altitude_feet     object
pilot_front                   object
pilot_rear                    object
data_ac                       object
data_adp                      object
data_imu                      object
data_kml                      object
data_uv                       object
dtype: object

In [17]:
rec = csv[csv['flight_number'] == 4]
print(rec)

  post_type   post_title post_content post_excerpt  flight_number  \
3    flight  Flight 0004                                        4   

  takeoff_date_local takeoff_time_local_HM takeoff_airport takeoff_city  \
3         2016-01-27                  1423            KMEV   Minden, NV   

  landing_date_local  ... release_altitude_feet maximum_altitude_feet  \
3         2016-01-27  ...                  7600                  7600   

  maximum_gps_altitude_feet  pilot_front  pilot_rear  data_ac data_adp  \
3                              Jim Payne                                 

  data_imu data_kml data_uv  
3                            

[1 rows x 24 columns]


In [18]:
csv.columns

Index(['post_type', 'post_title', 'post_content', 'post_excerpt',
       'flight_number', 'takeoff_date_local', 'takeoff_time_local_HM',
       'takeoff_airport', 'takeoff_city', 'landing_date_local',
       'landing_time_local_HM', 'landing_airport', 'landing_city',
       'flight_duration_hours', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_adp', 'data_imu', 'data_kml', 'data_uv'],
      dtype='object')

# Find Data Files

In [19]:
flts = pu.get_subdirs(flights_root)
#flts

In [20]:
# Walk the Flights dir, looking for data files, and put them in the CSV
def stuff_file_paths(df=None, flight_dirs=None, host="http://localhost", verbose=1):
    for flt in flight_dirs:
        files_full = pu.get_files(flt)
        files = [f[f.rfind('/')+1:] for f in files_full]
        if files == []:
            continue
        
        nr = int(flt[-4:])
        if verbose:
            print(f"#{nr}\t{flt}\t{files}")

        for file in files_full:
            file = f'{host}/{file}'
            base = file[:file.rfind('.')]
            suffix = file[file.rfind('.')+1:]
            idx = nr - 1
            #if verbose:
            #    print(f"base = {base}   suffix = {suffix}")
            if suffix == 'kml':
                df.loc[idx, 'data_kml'] = file
                if verbose:
                    print(f"KML = {file}")
            #print(f"base[-2:]={base[-2:]}")                    
            for kind in ['AC', 'ADP', 'IMU', 'UV']:
                if base[-len(kind):] == kind:
                    df.loc[idx, f'data_{kind.lower()}'] = file
                    if verbose:
                        print(f"{kind} = {file}")                
    return df

stuff_file_paths(df=csv, flight_dirs=flts)

#61	data/Flights/0061	['Flt0061IMU.zip', 'Flt0061AC.xlsb', 'Flt0061.kml', 'Flt0061ADP.csv', 'Flt0061UV.xlsx']
IMU = http://localhost/data/Flights/0061/Flt0061IMU.zip
AC = http://localhost/data/Flights/0061/Flt0061AC.xlsb
KML = http://localhost/data/Flights/0061/Flt0061.kml
ADP = http://localhost/data/Flights/0061/Flt0061ADP.csv
UV = http://localhost/data/Flights/0061/Flt0061UV.xlsx
#59	data/Flights/0059	['Flt0059AC.xlsx', 'Flt0059IMU.zip', 'Flt0059ADP.csv', 'Flt0059.kml', 'Flt0059UV.xlsx']
AC = http://localhost/data/Flights/0059/Flt0059AC.xlsx
IMU = http://localhost/data/Flights/0059/Flt0059IMU.zip
ADP = http://localhost/data/Flights/0059/Flt0059ADP.csv
KML = http://localhost/data/Flights/0059/Flt0059.kml
UV = http://localhost/data/Flights/0059/Flt0059UV.xlsx
#50	data/Flights/0050	['Flt0050UV.xlsx', 'Flt0050.kml', 'Flt0050AC.xlsb']
UV = http://localhost/data/Flights/0050/Flt0050UV.xlsx
KML = http://localhost/data/Flights/0050/Flt0050.kml
AC = http://localhost/data/Flights/0050/Flt0050A

,post_type,post_title,post_content,post_excerpt,flight_number,takeoff_date_local,takeoff_time_local_HM,takeoff_airport,takeoff_city,landing_date_local,...,release_altitude_feet,maximum_altitude_feet,maximum_gps_altitude_feet,pilot_front,pilot_rear,data_ac,data_adp,data_imu,data_kml,data_uv
0,flight,Flight 0001,First Flight,First Flight,1,2015-09-23,0818,KRDM,"Redmond, OR",2015-09-23,...,8100,8100,,Jim Payne,Morgan Sandercock,,,,,
1,flight,Flight 0002,flight testing,flight testing,2,2016-01-15,1307,KMEV,"Minden, NV",2016-01-15,...,10800,10800,,Jim Payne,Miguel Iturmendi,,,,,
2,flight,Flight 0003,,,3,2016-01-15,1420,KMEV,"Minden, NV",2016-01-15,...,10700,10700,,Jim Payne,Miguel Iturmendi,,,,,
3,flight,Flight 0004,,,4,2016-01-27,1423,KMEV,"Minden, NV",2016-01-27,...,7600,7600,,Jim Payne,,,,,,
4,flight,Flight 0005,,,5,2016-01-27,1525,KMEV,"Minden, NV",2016-01-27,...,8700,8700,,Jim Payne,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,flight,Flight 0061,,,61,2019-08-31,1037,SAWC,El Calafate,2019-08-31,...,46600,56300,,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0061/Flt0061AC.xlsb,http://localhost/data/Flights/0061/Flt0061ADP.csv,http://localhost/data/Flights/0061/Flt0061IMU.zip,http://localhost/data/Flights/0061/Flt0061.kml,http://localhost/data/Flights/0061/Flt0061UV.xlsx
61,flight,Flight 0062,,,62,2019-09-06,1333,SAWC,El Calafate,2019-09-06,...,42000,42300,,Jim Payne,Morgan Sandercock,http://localhost/data/Flights/0062/Flt0062AC.xlsb,http://localhost/data/Flights/0062/Flt0062ADP.csv,http://localhost/data/Flights/0062/Flt0062IMU.zip,http://localhost/data/Flights/0062/Flt0062.kml,http://localhost/data/Flights/0062/Flt0062UV.xlsx
62,flight,Flight 0063,tow height record,tow height record,63,2019-09-11,1235,SAWC,El Calafate,2019-09-11,...,47100,50600,,Jim Payne,Tim Gardner,http://localhost/data/Flights/0063/Flt0063AC.xlsb,http://localhost/data/Flights/0063/Flt0063ADP.zip,http://localhost/data/Flights/0063/Flt0063IMU.zip,http://localhost/data/Flights/0063/Flt0063.kml,http://localhost/data/Flights/0063/Flt0063UV.xlsb
63,flight,Flight 0064,,,64,2019-09-14,0913,SAWC,El Calafate,2019-09-14,...,45100,49200,,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0064/Flt0064AC.xlsb,http://localhost/data/Flights/0064/Flt0064ADP.csv,http://localhost/data/Flights/0064/Flt0064IMU.zip,http://localhost/data/Flights/0064/Flt0064.kml,http://localhost/data/Flights/0064/Flt0064UV.xlsb


# Sanity Check: Output

In [21]:
csv.iloc[0]

post_type                               flight
post_title                         Flight 0001
post_content                      First Flight
post_excerpt                      First Flight
flight_number                                1
takeoff_date_local                  2015-09-23
takeoff_time_local_HM                     0818
takeoff_airport                           KRDM
takeoff_city                       Redmond, OR
landing_date_local                  2015-09-23
landing_time_local_HM                     0851
landing_airport                           KRDM
landing_city                       Redmond, OR
flight_duration_hours                      0.6
release_altitude_feet                     8100
maximum_altitude_feet                     8100
maximum_gps_altitude_feet                     
pilot_front                          Jim Payne
pilot_rear                   Morgan Sandercock
data_ac                                       
data_adp                                      
data_imu     

In [22]:
csv.iloc[64]

post_type                                                               flight
post_title                                                         Flight 0065
post_content                                               Last flight of 2019
post_excerpt                                               Last flight of 2019
flight_number                                                               65
takeoff_date_local                                                  2019-09-17
takeoff_time_local_HM                                                     1120
takeoff_airport                                                           SAWC
takeoff_city                                                       El Calafate
landing_date_local                                                  2019-09-17
landing_time_local_HM                                                     1650
landing_airport                                                           SAWC
landing_city                                        

In [23]:
csv.columns

Index(['post_type', 'post_title', 'post_content', 'post_excerpt',
       'flight_number', 'takeoff_date_local', 'takeoff_time_local_HM',
       'takeoff_airport', 'takeoff_city', 'landing_date_local',
       'landing_time_local_HM', 'landing_airport', 'landing_city',
       'flight_duration_hours', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_adp', 'data_imu', 'data_kml', 'data_uv'],
      dtype='object')

# Write Generic CSV
This CSV is not Toolset-specific (see below)

In [24]:
# Generic CSV

csv.to_csv(f"{data_root}/flights.csv", index=False)
print("generic CSV saved!")

generic CSV saved!


# CSV for Toolset
For import in the Toolset plugin Types module 
- Toolset requires custom post fields prepended with `wpcf-` (the WPCF plugin was Toolset's first?)
- Date/time fields in POSIX timestamp format (float)

In [25]:
df = csv.copy()

In [26]:
df.columns

Index(['post_type', 'post_title', 'post_content', 'post_excerpt',
       'flight_number', 'takeoff_date_local', 'takeoff_time_local_HM',
       'takeoff_airport', 'takeoff_city', 'landing_date_local',
       'landing_time_local_HM', 'landing_airport', 'landing_city',
       'flight_duration_hours', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_adp', 'data_imu', 'data_kml', 'data_uv'],
      dtype='object')

## Convert date/time to POSIX timestamp format
Well, it's unfortunate that Toolset uses POSIX timestamp objects, as you can't just eyeball them.  

Python has very powerful datetime manipulation, used here for converting a string format to timestmp.  It is necessary to account for timezone in how you use Python datetime.  Best practices call for date and time to be stored as Zulu time (UTC) and convert to local time when needed to display.  However, the Perlan times were given to me as local time, and the timezones are not the same for all airports we've used.  Though it might be possible to store as UTC in WordPress and convert to what the local time was at the time of the flight, this seems like way more effort than it is worth, especially with daylight saving time!  And given WordPress, I'm not certain that's even possible in the Toolset Types module.  Thus I force the creation of datetime objects with zero offset from UTC, done here with the `%z` in the format string, and "+0000" in the input string, effectively using them as "naive" datetime objects (tho not exactly - it's complicated).

In [27]:
df[f'takeoff_datetime_local_YmdHM'] = df.apply(lambda row: f"{row.takeoff_date_local}-{row.takeoff_time_local_HM}", axis=1)
df['takeoff_datetime_local'] = df['takeoff_datetime_local_YmdHM'].apply(lambda x: dt.strptime(x + "+0000", "%Y-%m-%d-%H%M%z").timestamp())

df[f'landing_datetime_local_YmdHM'] = df.apply(lambda row: f"{row.landing_date_local}-{row.landing_time_local_HM}", axis=1)
df['landing_datetime_local'] = df['landing_datetime_local_YmdHM'].apply(lambda x: dt.strptime(x + "+0000", "%Y-%m-%d-%H%M%z").timestamp())

In [28]:
df.iloc[64]

post_type                                                                  flight
post_title                                                            Flight 0065
post_content                                                  Last flight of 2019
post_excerpt                                                  Last flight of 2019
flight_number                                                                  65
takeoff_date_local                                                     2019-09-17
takeoff_time_local_HM                                                        1120
takeoff_airport                                                              SAWC
takeoff_city                                                          El Calafate
landing_date_local                                                     2019-09-17
landing_time_local_HM                                                        1650
landing_airport                                                              SAWC
landing_city    

## Prepend 'wpcf-'

In [29]:
# prepend 'wpcf-'
for col in df.columns:
    if col[:len('post_')] == 'post_':
        continue
    else:
        df = df.rename(columns={col : f"wpcf-{col}"})    # note the dash in 'wpcf-'

## Make all column names lower-case
Toolset requires this, apparently.

In [30]:
for col in df.columns:
    df = df.rename(columns={col : col.lower()})

In [31]:
df.columns

Index(['post_type', 'post_title', 'post_content', 'post_excerpt',
       'wpcf-flight_number', 'wpcf-takeoff_date_local',
       'wpcf-takeoff_time_local_hm', 'wpcf-takeoff_airport',
       'wpcf-takeoff_city', 'wpcf-landing_date_local',
       'wpcf-landing_time_local_hm', 'wpcf-landing_airport',
       'wpcf-landing_city', 'wpcf-flight_duration_hours',
       'wpcf-release_altitude_feet', 'wpcf-maximum_altitude_feet',
       'wpcf-maximum_gps_altitude_feet', 'wpcf-pilot_front', 'wpcf-pilot_rear',
       'wpcf-data_ac', 'wpcf-data_adp', 'wpcf-data_imu', 'wpcf-data_kml',
       'wpcf-data_uv', 'wpcf-takeoff_datetime_local_ymdhm',
       'wpcf-takeoff_datetime_local', 'wpcf-landing_datetime_local_ymdhm',
       'wpcf-landing_datetime_local'],
      dtype='object')

In [32]:
df.iloc[0]

post_type                                       flight
post_title                                 Flight 0001
post_content                              First Flight
post_excerpt                              First Flight
wpcf-flight_number                                   1
wpcf-takeoff_date_local                     2015-09-23
wpcf-takeoff_time_local_hm                        0818
wpcf-takeoff_airport                              KRDM
wpcf-takeoff_city                          Redmond, OR
wpcf-landing_date_local                     2015-09-23
wpcf-landing_time_local_hm                        0851
wpcf-landing_airport                              KRDM
wpcf-landing_city                          Redmond, OR
wpcf-flight_duration_hours                         0.6
wpcf-release_altitude_feet                        8100
wpcf-maximum_altitude_feet                        8100
wpcf-maximum_gps_altitude_feet                        
wpcf-pilot_front                             Jim Payne
wpcf-pilot

In [33]:
# Toolset-specific CSV

df.to_csv(f"{data_root}/flights-toolset.csv", index=False)
print("Toolset CSV saved!")

Toolset CSV saved!
